In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import os
import coco
import yolo
import vgg

In [2]:
repo_dir = project_name = 'COTSDataset' 
part1_single_objects = os.path.join(repo_dir, "Part 1 - Single Objects")
part2_multiple_objects = os.path.join(repo_dir, "Part 2 - Multiple Objects")
part3_complex_background = os.path.join(repo_dir, "Part 3 - Complex Background")

In [10]:
do_display = True
do_save = False
do_print = False

# COCO
coco_output_dir = "COCO_annotations"
coco_color = (0, 255, 0)

# YOLO
yolo_output_dir = "YOLO_annotations"
yolo_color = (0, 255, 0)

# VGG
vgg_output_dir = "VGG_annotations"
vgg_color = 'r'

In [11]:
def iou(annotation):
    gt = annotation['image']
    annotation_mask = np.zeros((annotation['height'], annotation['width']), dtype=np.uint8)
    cv2.drawContours(annotation_mask, annotation['contours'], -1, 255, -1)
    # resize annotation mask to gt size
    annotation_mask = cv2.resize(annotation_mask, (gt.shape[1], gt.shape[0]))

    # plot both on each other transparently, one blue one red
    # plt.figure()
    # plt.rcParams['figure.figsize'] = [20, 10]
    # plt.imshow(gt, cmap='Blues', alpha=0.5)
    # plt.imshow(annotation_mask, cmap='Reds', alpha=0.5)
    # plt.xticks([])
    # plt.yticks([])
    # plt.show()

    intersection = np.logical_and(gt, annotation_mask)
    union = np.logical_or(gt, annotation_mask)
    return np.sum(intersection) / np.sum(union)

In [12]:
# Import coco
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
import skimage.io as io

In [4]:
ann_file = 'Evaluation/instances_train2017.json'
coco_data = COCO(ann_file)

loading annotations into memory...
Done (t=20.96s)
creating index...
index created!


In [13]:
coco_annotations = coco_data.loadAnns(coco_data.getAnnIds())

In [ ]:
# coco_annotations = coco_data.loadAnns(coco_data.getAnnIds())

# # Merge annotations with same image id
# merged_annotations = {}
# for annotation in coco_annotations:
#     # print(merged_annotations)
#     if annotation['image_id'] in merged_annotations:
#         merged_annotations[annotation['image_id']]['segmentation'].extend(annotation['segmentation'])
#     else:
#         merged_annotations[annotation['image_id']] = annotation

In [9]:
from random import randint
rand_id = randint(0, len(coco_annotations))

In [24]:
rand_id = 555669

In [25]:
id_ = coco_annotations[rand_id]['image_id']
name = coco_data.loadImgs(id_)[0]['file_name']
image = coco_data.annToMask(coco_annotations[rand_id])

original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])


im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

annotation = coco.annotate(im, do_display=True, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.POLY_APPROX)

contours = annotation['contours']

annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)


 Annotating image:  000000034381.jpg
------------------------------------------------------------------------------------------------------------------------


c:\Work\uni\Summer Placement\mask-to-annotation\coco.py:60: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [16]:
import matplotlib.pyplot as plt

In [26]:
original_mask = image

plt.figure()
plt.rcParams['figure.figsize'] = [20, 10]
plt.subplot(131)
plt.imshow(original)
plt.xticks([])
plt.yticks([])
plt.title('Original Image', fontsize=20)
plt.subplot(132)
image_rgba = cv2.cvtColor(original_mask, cv2.COLOR_GRAY2RGBA)
alpha = np.where(image == 0, 0, 255)
image_rgba[:, :, 0] = 0
image_rgba[:, :, 1] = 0
image_rgba[:, :, 2] = 255 
image_rgba[:, :, 3] = alpha

plt.imshow(original, alpha=1)
plt.imshow(image_rgba, alpha=0.5)

plt.xticks([])
plt.yticks([])
plt.title('Original Mask', fontsize=20)
plt.subplot(133)
# plot contours
plt.imshow(original_mask, cmap='gray')
# plt.imshow(annotated_image)
annotated_rgba = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGBA)
alpha = np.where(annotated_image == 0, 0, 255)
annotated_rgba[:, :, 0] = 255
annotated_rgba[:, :, 1] = 0
annotated_rgba[:, :, 2] = 0
annotated_rgba[:, :, 3] = alpha[:, :, 0]
plt.imshow(annotated_rgba, alpha=1)


plt.xticks([])
plt.yticks([])
plt.title('Annotated Mask', fontsize=20)
plt.show()
plt.savefig('test.jpeg')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [44]:
cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [ ]:
annotation['contours']

[]

**IoU**

In [15]:
images = {}
ids = []

**PA**

In [108]:
len(images)

168

In [16]:
from time import time

start = time()

image_id = 0

epsilon = 0.005

coco_pa_ious = []

for i in range(200):
    id_ = coco_annotations[i]['image_id']

    if id_ not in ids:
        name = coco_data.loadImgs(id_)[0]['file_name']
        ids.append(id_)

        image = coco_data.annToMask(coco_annotations[id_])
        
        original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
        images[id_] = original

        im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

        annotation = coco.annotate(im, do_display=False, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.POLY_APPROX)

        contours = annotation['contours']

        annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
        cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

        image_id += 1
        coco_pa_ious.append([name, iou(annotation)])

end = time()
print(f'PA time: {end - start}')


 Annotating image:  000000558840.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000495357.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000116061.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000016164.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000205350.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000000074.jpg
--------

In [128]:
# Plot bar chart of ious, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 75]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_pa_ious)), [x[1] for x in coco_pa_ious], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_pa_ious]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_pa_ious)), [x[0] for x in coco_pa_ious], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_pa_ious]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_pa_ious]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average IoU (' + str(np.round(np.mean([x[1] for x in coco_pa_ious]), 3)) + ')'
median_legend = 'Median IoU (' + str(np.round(np.median([x[1] for x in coco_pa_ious]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('IoU', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('IoU per Image - COCO (Polygon Approximation)', fontsize=20, fontweight='bold')

plt.show()
plt.savefig('./Figures/IoU per Image - COCO (Polygon Approximation).png')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**KMC**

In [116]:
coco_kmc_ious

[]

In [129]:
len(ids)

168

In [17]:
image_id = 0

epsilon = 0.005

coco_kmc_ious = []

for id_ in ids:
    name = coco_data.loadImgs(id_)[0]['file_name']

    image = coco_data.annToMask(coco_annotations[id_])
    
    # original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
    original = images[id_]

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    annotation = coco.annotate(im, do_display=False, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.K_MEANS_CLUSTER)

    contours = annotation['contours']

    annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
    cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

    image_id += 1
    coco_kmc_ious.append([name, iou(annotation)])


 Annotating image:  000000558840.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000495357.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000116061.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000016164.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000205350.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000000074.jpg
--------

In [18]:
# Plot bar chart of ious, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 75]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_kmc_ious)), [x[1] for x in coco_kmc_ious], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_kmc_ious]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_kmc_ious)), [x[0] for x in coco_kmc_ious], fontsize=10)

# make x axis span 0 to 1
plt.xlim(0, 1)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_kmc_ious]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_kmc_ious]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average IoU (' + str(np.round(np.mean([x[1] for x in coco_kmc_ious]), 3)) + ')'
median_legend = 'Median IoU (' + str(np.round(np.median([x[1] for x in coco_kmc_ious]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('IoU', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('IoU per Image - COCO (K-Means Clustering)', fontsize=20, fontweight='bold')

plt.show()
plt.savefig('./Figures/IoU per Image - COCO (K-Means Clustering).png')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**SAM**

In [19]:
# Dependencies required for segment-anything
# %pip install git+https://github.com/facebookresearch/segment-anything.git
# %pip install torch torchvision
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch
torch.cuda.empty_cache()
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Segment-anything model
# Downloading model checkpoint:
# Utilise the following link: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
# Place the downloaded checkpoint in a new directory named Sam_checkpoints
sam_checkpoint = "Sam_checkpoints/sam_vit_b_01ec64.pth"
model_type = "vit_b"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_batch=16
)

In [20]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [24]:
ids[67]

555669

In [25]:
masks = {}

from time import time
start = time()

for id_ in ids[68:]:
    name = coco_data.loadImgs(id_)[0]['file_name']

    image = coco_data.annToMask(coco_annotations[id_])
    
    # original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])

    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    mask = mask_generator.generate(image)

    sorted_anns = sorted(mask, key=(lambda x: x['area']), reverse=True)
    img = np.zeros((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))

    for ann in sorted_anns:
        # skip first anotation
        if ann is sorted_anns[0]:
            continue
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = 1

    img[:, :, 3] = 1

    # save
    plt.imsave(f'Evaluation/COCO/segment-anything/{name}', img)

    masks[name] = mask
    # masks.append(mask)
    print(f'SAM Mask generated for {name} -- Time elapsed: {time() - start}')

end = time()
print(f'Total time elapsed: {end - start}')


SAM Mask generated for 000000270744.jpg -- Time elapsed: 16.154730081558228
SAM Mask generated for 000000351840.jpg -- Time elapsed: 23.64141345024109
SAM Mask generated for 000000307993.jpg -- Time elapsed: 32.60654854774475
SAM Mask generated for 000000145544.jpg -- Time elapsed: 39.35654616355896
SAM Mask generated for 000000184771.jpg -- Time elapsed: 48.633782625198364
SAM Mask generated for 000000187349.jpg -- Time elapsed: 57.316757678985596
SAM Mask generated for 000000282134.jpg -- Time elapsed: 65.60446286201477
SAM Mask generated for 000000378561.jpg -- Time elapsed: 72.0874617099762
SAM Mask generated for 000000007125.jpg -- Time elapsed: 78.67846131324768
SAM Mask generated for 000000386165.jpg -- Time elapsed: 87.48154258728027
SAM Mask generated for 000000346965.jpg -- Time elapsed: 94.96954274177551
SAM Mask generated for 000000559950.jpg -- Time elapsed: 102.7700686454773
SAM Mask generated for 000000459566.jpg -- Time elapsed: 111.80018615722656
SAM Mask generated for

In [39]:
masks['000000270744.jpg'][0]

{'segmentation': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 'area': 270760,
 'bbox': [0, 2, 639, 423],
 'predicted_iou': 1.0405972003936768,
 'point_coords': [[490.0, 153.453125]],
 'stability_score': 0.9976194500923157,
 'crop_box': [0, 0, 640, 427]}

In [32]:
coco_annotations[id_]

{'segmentation': [[427.76,
   162.28,
   433.3,
   163.51,
   438.42,
   166.79,
   444.78,
   168.43,
   448.47,
   176.43,
   450.72,
   181.14,
   451.34,
   182.78,
   454.21,
   181.96,
   451.75,
   172.53,
   451.54,
   169.45,
   454.21,
   168.43,
   462.0,
   169.25,
   465.9,
   170.89,
   471.23,
   173.76,
   477.17,
   183.4,
   477.58,
   188.11,
   475.53,
   188.73,
   473.28,
   180.32,
   468.97,
   174.38,
   469.59,
   181.96,
   476.76,
   193.44,
   477.79,
   195.9,
   474.3,
   196.72,
   472.66,
   191.8,
   465.69,
   184.22,
   465.28,
   191.39,
   460.77,
   198.77,
   454.21,
   198.57,
   454.21,
   195.29,
   459.34,
   194.67,
   461.8,
   189.75,
   459.95,
   183.6,
   441.5,
   184.22,
   439.24,
   198.77,
   433.09,
   198.77,
   432.89,
   196.11,
   436.58,
   194.26,
   437.4,
   185.24,
   432.07,
   186.88,
   426.33,
   195.49,
   424.07,
   193.03,
   432.07,
   184.01,
   434.53,
   178.07,
   431.25,
   171.51,
   423.45,
   176.43,
   42

In [51]:
masks[name][0]

{'segmentation': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 'area': 270760,
 'bbox': [0, 2, 639, 423],
 'predicted_iou': 1.0405972003936768,
 'point_coords': [[490.0, 153.453125]],
 'stability_score': 0.9976194500923157,
 'crop_box': [0, 0, 640, 427]}

In [62]:
masks[name][0]

{'segmentation': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 'area': 270760,
 'bbox': [0, 2, 639, 423],
 'predicted_iou': 1.0405972003936768,
 'point_coords': [[490.0, 153.453125]],
 'stability_score': 0.9976194500923157,
 'crop_box': [0, 0, 640, 427],
 'image_id': 270744}

In [64]:
for mask in masks:
    print(mask['segmentation'].shape)

TypeError: string indices must be integers

In [61]:
coco_annotations[id_]

{'segmentation': [[427.76,
   162.28,
   433.3,
   163.51,
   438.42,
   166.79,
   444.78,
   168.43,
   448.47,
   176.43,
   450.72,
   181.14,
   451.34,
   182.78,
   454.21,
   181.96,
   451.75,
   172.53,
   451.54,
   169.45,
   454.21,
   168.43,
   462.0,
   169.25,
   465.9,
   170.89,
   471.23,
   173.76,
   477.17,
   183.4,
   477.58,
   188.11,
   475.53,
   188.73,
   473.28,
   180.32,
   468.97,
   174.38,
   469.59,
   181.96,
   476.76,
   193.44,
   477.79,
   195.9,
   474.3,
   196.72,
   472.66,
   191.8,
   465.69,
   184.22,
   465.28,
   191.39,
   460.77,
   198.77,
   454.21,
   198.57,
   454.21,
   195.29,
   459.34,
   194.67,
   461.8,
   189.75,
   459.95,
   183.6,
   441.5,
   184.22,
   439.24,
   198.77,
   433.09,
   198.77,
   432.89,
   196.11,
   436.58,
   194.26,
   437.4,
   185.24,
   432.07,
   186.88,
   426.33,
   195.49,
   424.07,
   193.03,
   432.07,
   184.01,
   434.53,
   178.07,
   431.25,
   171.51,
   423.45,
   176.43,
   42

In [73]:
coco_sam_ious = []

for id_ in ids:
    # id_ = coco_annotations[i]['image_id']
    name = coco_data.loadImgs(id_)[0]['file_name']

    image = coco_data.annToMask(coco_annotations[id_])
    
    original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
    # original = images[id_]

    annotation = cv2.imread('./Evaluation/COCO/segment-anything/' + name)
    annotation = cv2.cvtColor(annotation, cv2.COLOR_BGR2GRAY)

    # annotation = mask_generator.generate(original)
        
    intersection = np.logical_and(image, annotation)
    union = np.logical_or(image, annotation)
    iou_score = np.sum(intersection) / np.sum(union)

    coco_sam_ious.append([name, iou_score])

    print(f'{name}:{iou_score}')

    # Comparison
    original_mask = image
    # original_mask = cv2.resize  (original_mask, (original.shape[1], original.shape[0]))

    plt.figure()
    plt.rcParams['figure.figsize'] = [20, 10]
    plt.subplot(131)
    plt.imshow(original)
    plt.xticks([])
    plt.yticks([])
    plt.title('Original Image', fontsize=20)
    plt.subplot(132)
    image_rgba = cv2.cvtColor(original_mask, cv2.COLOR_GRAY2RGBA)
    alpha = np.where(image == 0, 0, 255)
    image_rgba[:, :, 0] = 0
    image_rgba[:, :, 1] = 0
    image_rgba[:, :, 2] = 255 
    image_rgba[:, :, 3] = alpha
    image_rgba = cv2.resize(image_rgba, (original.shape[1], original.shape[0]))

    plt.imshow(original, alpha=1)
    plt.imshow(image_rgba, alpha=0.5)

    plt.xticks([])
    plt.yticks([])
    plt.title('Original Mask', fontsize=20)
    plt.subplot(133)
    # plot contours
    plt.imshow(original_mask, cmap='gray')
    # plt.imshow(annotated_image)
    annotated_rgba = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGBA)
    alpha = np.where(annotated_image == 0, 0, 255)
    annotated_rgba[:, :, 0] = 255
    annotated_rgba[:, :, 1] = 0
    annotated_rgba[:, :, 2] = 0
    annotated_rgba[:, :, 3] = alpha[:, :, 0]

    annotated_rgba = cv2.resize(annotated_rgba, (original.shape[1], original.shape[0]))

    plt.imshow(annotation, alpha=1)


    plt.xticks([])
    plt.yticks([])
    plt.title('Annotated Mask', fontsize=20)
    plt.show()
    plt.savefig('./Evaluation/COCO/segment-anything/comparisons/' + name)


000000558840.jpg:0.7413441955193483


c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


000000200365.jpg:0.6291512915129152
000000495357.jpg:0.9609868805438997
000000116061.jpg:0.16739947691756868
000000016164.jpg:0.8042468480424685
000000205350.jpg:0.015118167057550064
000000000074.jpg:0.7407444900792503
000000212545.jpg:0.7205793602896802
000000514915.jpg:0.7952468007312614
000000154589.jpg:0.7761461318051576
000000471175.jpg:0.0
000000225919.jpg:0.7862306368330465
000000400728.jpg:0.924424741397509
000000194306.jpg:0.826362484157161
000000383780.jpg:0.7198992443324937
000000580255.jpg:0.6661059714045416
000000370210.jpg:0.921100583090379
000000075283.jpg:0.0
000000325969.jpg:0.7360980207351555
000000251716.jpg:0.0
000000013882.jpg:0.7339901477832512
000000185156.jpg:0.6351931330472103
000000176697.jpg:0.0
000000376608.jpg:0.3641304347826087
000000178939.jpg:0.07911004361291954
000000173350.jpg:0.8752886836027713
000000026654.jpg:0.0
000000346071.jpg:0.9676513253329817
000000158497.jpg:0.6503613118399111
000000408307.jpg:0.750690563371234
000000252203.jpg:0.715907767555

In [74]:
# Plot bar chart of ious, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 75]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_sam_ious)), [x[1] for x in coco_sam_ious], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_sam_ious]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_sam_ious)), [x[0] for x in coco_sam_ious], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_sam_ious]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_sam_ious]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average IoU (' + str(np.round(np.mean([x[1] for x in coco_sam_ious]), 3)) + ')'
median_legend = 'Median IoU (' + str(np.round(np.median([x[1] for x in coco_sam_ious]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('IoU', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('IoU per Image (Segment Anything - COCO)', fontsize=20, fontweight='bold')

plt.show()
plt.savefig('./Figures/IoU per Image - COCO (Segment Anything).png')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**Compactness**

In [132]:
# compactness
def compactness(annotation):
    mask = cv2.drawContours(np.zeros((annotation['height'], annotation['width']), dtype=np.uint8), annotation['contours'], -1, 255, -1)
    bounding_rect = cv2.boundingRect(mask)
    box = bounding_rect[2] * bounding_rect[3]
    
    area = np.sum(mask != 0)

    return area / box

In [1]:
# def compactness(annotation):
#     # area of annotation
#     area = annotation['image'].shape[0] * annotation['image'].shape[1]

#     # perimeter of annotation
#     perimeter = 0
#     for contour in annotation['contours']:
#         perimeter += cv2.arcLength(contour, True)

#     return (perimeter ** 2) / area

**PA**

In [133]:
from time import time

start = time()

epsilon = 0.005

coco_pa_compactness = []

for id_ in ids:
    name = coco_data.loadImgs(id_)[0]['file_name']
    image = coco_data.annToMask(coco_annotations[id_])
    
    # original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
    original = images[id_]

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    annotation = coco.annotate(im, do_display=False, do_save=True, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.POLY_APPROX)

    score = compactness(annotation)
    if score < np.inf:
        coco_pa_compactness.append([name, score])

end = time()
print(f'PA time: {end - start}')


 Annotating image:  000000558840.jpg
 Succesfully saved image:  000000558840.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
 Succesfully saved image:  000000200365.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000495357.jpg
 Succesfully saved image:  000000495357.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000116061.jpg
 Succesfully saved image:  000000116061.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000016164.jpg
 Succesfully saved image:  000000016164.jpg 


-------------------------------------------------------------------------------------------

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


 000000047263.jpg
 Succesfully saved image:  000000047263.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000519838.jpg
 Succesfully saved image:  000000519838.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000283119.jpg
 Succesfully saved image:  000000283119.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000369190.jpg
 Succesfully saved image:  000000369190.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000458424.jpg
 Succesfully saved image:  000000458424.jpg 


---------------------------------------------------------------------------------------------------------------

In [134]:
# Plot bar chart of compactness, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 35]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_pa_compactness)), [x[1] for x in coco_pa_compactness], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_pa_compactness]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_pa_compactness)), [x[0] for x in coco_pa_compactness], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_pa_compactness]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_pa_compactness]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average Compactness (' + str(np.round(np.mean([x[1] for x in coco_pa_compactness]), 3)) + ')'
median_legend = 'Median Compactness (' + str(np.round(np.median([x[1] for x in coco_pa_compactness]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('Compactness', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('Compactness per Image (Polygon Approximation)', fontsize=20, fontweight='bold')

plt.show()

plt.savefig('./Figures/Compactness per Image - COCO (Polygon Approximation).png')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**KMC**

In [135]:
from time import time

start = time()

epsilon = 0.005

coco_kmc_compactness = []

for id_ in ids:
    name = coco_data.loadImgs(id_)[0]['file_name']
    image = coco_data.annToMask(coco_annotations[id_])
    
    # original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
    original = images[id_]

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    annotation = coco.annotate(im, do_display=False, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.K_MEANS_CLUSTER)

    score = compactness(annotation)
    if score < np.inf:
        coco_kmc_compactness.append([name, score])

end = time()
print(f'PA time: {end - start}')


 Annotating image:  000000558840.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000495357.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000116061.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000016164.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000205350.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000000074.jpg
--------

In [136]:
# Plot bar chart of compactness, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 35]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_kmc_compactness)), [x[1] for x in coco_kmc_compactness], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_kmc_compactness]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_kmc_compactness)), [x[0] for x in coco_kmc_compactness], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_kmc_compactness]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_kmc_compactness]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average Compactness (' + str(np.round(np.mean([x[1] for x in coco_kmc_compactness]), 3)) + ')'
median_legend = 'Median Compactness (' + str(np.round(np.median([x[1] for x in coco_kmc_compactness]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('Compactness', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('Compactness per Image (Polygon Approximation)', fontsize=20, fontweight='bold')

plt.show()

plt.savefig('./Figures/Compactness per Image - COCO (K-Means Clustering).png')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [87]:
images[75283]

array([[[77, 73, 72],
        [76, 72, 71],
        [76, 72, 71],
        ...,
        [56, 51, 55],
        [55, 50, 54],
        [56, 51, 55]],

       [[75, 71, 70],
        [75, 71, 70],
        [76, 72, 71],
        ...,
        [56, 51, 55],
        [55, 50, 54],
        [55, 50, 54]],

       [[77, 73, 74],
        [77, 73, 72],
        [79, 75, 74],
        ...,
        [54, 52, 55],
        [54, 52, 55],
        [53, 51, 54]],

       ...,

       [[41, 41, 43],
        [37, 36, 41],
        [37, 38, 43],
        ...,
        [63, 63, 61],
        [57, 57, 55],
        [27, 28, 23]],

       [[36, 36, 38],
        [33, 32, 37],
        [33, 34, 39],
        ...,
        [65, 65, 63],
        [62, 62, 60],
        [53, 54, 49]],

       [[34, 34, 36],
        [35, 34, 39],
        [35, 36, 41],
        ...,
        [71, 71, 69],
        [60, 60, 58],
        [62, 63, 58]]], dtype=uint8)

In [92]:

name = coco_data.loadImgs(75283)[0]['file_name']
image = coco_data.annToMask(coco_annotations[75283])

# original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
original = images[75283]

im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

annotation = coco.annotate(im, do_display=False, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.K_MEANS_CLUSTER)


 Annotating image:  000000075283.jpg
------------------------------------------------------------------------------------------------------------------------


**SAM**

In [137]:
image_id = 0

coco_sam_compactness = []

start = time()

for id_ in ids:
    name = coco_data.loadImgs(id_)[0]['file_name']
    image = coco_data.annToMask(coco_annotations[id_])
    
    # original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])
    original = images[id_]

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    mask = cv2.imread('./Evaluation/COCO/segment-anything/' + name)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    bounding_rect = cv2.boundingRect(mask)
    box = bounding_rect[2] * bounding_rect[3]
    
    area = np.sum(mask != 0)

    score = area / box
    if score < np.inf:
        coco_sam_compactness.append([name, score])

end = time()
print(f'KMC time: {end - start}')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in long_scalars


KMC time: 0.576115608215332


In [138]:
# Plot bar chart of compactness, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 35]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_sam_compactness)), [x[1] for x in coco_sam_compactness], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_sam_compactness]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_sam_compactness)), [x[0] for x in coco_sam_compactness], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_sam_compactness]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_sam_compactness]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average Compactness (' + str(np.round(np.mean([x[1] for x in coco_sam_compactness]), 3)) + ')'
median_legend = 'Median Compactness (' + str(np.round(np.median([x[1] for x in coco_sam_compactness]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('Compactness', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('Compactness per Image (Polygon Approximation)', fontsize=20, fontweight='bold')

plt.show()

plt.savefig('./Figures/Compactness per Image - COCO (Segment Anything).png')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
